In [ ]:
scale_cadence = 3

from pygeodyn.PYGEODYN import Pygeodyn
import datetime


scaling_cadence = 3
scale_cadence = scaling_cadence
index_buffarc_start = 0
run_list = [
                'msis2',
                'jb2008',
                'dtm2020_o',
                'hasdm_oc'
           ]

month_list = ['jan', 'feb', 'mar', 'apr','may', 'jun','jul','aug','sep', 'oct', 'nov','dec' ]
# 'jan', 'feb', 'mar', 'apr']#,
# month_list = ['apr']#,
# month_list = [ 'mar', 'apr']#,

# month_list = ['oct', 'nov','dec']



dir_modeldat='/data/SatDragModelValidation/data/inputs/atmos_models'
run_dict={}
for i in run_list:
    for y_num in [2018,2019]: #, 
        for imonth,month in enumerate(month_list):
            if y_num==2018 and month not in ['oct', 'nov', 'dec']:
                print(f'Skipping {y_num}.{month}')
                continue
            if y_num==2019 and month not in ['jan', 'feb', 'mar', 'apr']:
                print(f'Skipping {y_num}.{month}')
                continue

            if i =='msis2':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 5
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='dtm2020_o':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 3
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='jb2008':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 1
                run_dict[str(y_num)+month+i]['model_path'] = None
            if i =='hasdm_oc':
                run_dict[str(y_num)+month+i]={}
                run_dict[str(y_num)+month+i]['num'] = 2 
                run_dict[str(y_num)+month+i]['model_path'] = dir_modeldat+'/hasdm/sethasdm_orbitclouds' #HASDM_OrbitCloud_2018313.01.csv


print(run_dict)

In [ ]:

import pandas as pd
from datetime import datetime,timedelta
import numpy as np

# timestart = pd.to_datetime('2022-06-01 00:00:00', format='%Y-%m-%d %H:%M:%S')
# timeend   = pd.to_datetime('2022-06-30 00:00:00', format='%Y-%m-%d %H:%M:%S')
# scale_cadence = 3

# arcs_yyyyddd         = []
# epochstart = []
# epochstop   = []

# itime = timestart
# while itime < timeend:
               
#     itime_0 = itime
#     itime = itime + pd.to_timedelta(24,'h')
    
#     arcs_yyyyddd.append(itime_0.strftime('%Y.%j'))
#     epochstart.append(itime_0.strftime('%Y-%m-%d %H:%M:%S'))
#     epochstop.append(itime.strftime(  '%Y-%m-%d %H:%M:%S'))

# #     print(' ',itime_0.strftime('%Y.%j'),itime_0, ' to ', itime)        


# ##### update the epoch start to be 3-hours before and epoch end to be 3 hours later
# input_arcs       = []
# input_epochstart = []
# input_epochstop   = []


# for i in epochstart:
#     itime = pd.to_datetime(i) - pd.to_timedelta(3,'h')
#     input_epochstart.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

# for i in epochstop:
#     itime = pd.to_datetime(i) + pd.to_timedelta(3,'h')
#     input_epochstop.append(itime.strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
month_list

In [ ]:
import pandas as pd
from datetime import datetime,timedelta
import numpy as np



arc_timesfile = '/data/SatDragModelValidation/data/inputs/raw_inputdata/data_ICESat2/arc_times.txt'

### Make a function that finds the right range of dates in this file.
arcs = pd.read_csv(arc_timesfile, 
            sep = ',',
#             dtype=object,
            names = [
                'arc'         ,
                'epoch_start' ,
                'epoch_stop'  ,
                'orbit_start' ,
                'orbit_stop'  ,
                    ],)


arcs_yyyyddd = [x.strip() for x in arcs['arc'].values.tolist()]
epochstart   = [x.strip() for x in arcs['orbit_start'].values.tolist()]
epochstop    = [x.strip() for x in arcs['orbit_stop' ].values.tolist()]


input_arcs       = []
input_epochstart = []
input_epochstop  = []
arc_length = []
scaleparameter_times = []

for y_num in [2018, 2019]:

    for imonth,month in enumerate(month_list):
        if month=='jan':  m_num = 1
        if month=='feb':  m_num = 2
        if month=='mar':  m_num = 3
        if month=='apr':  m_num = 4
        if month=='may':  m_num = 5
        if month=='jun':  m_num = 6
        if month=='jul':  m_num = 7
        if month=='aug':  m_num = 8
        if month=='sep':  m_num = 9
        if month=='oct':  m_num = 10
        if month=='nov':  m_num = 11
        if month=='dec':  m_num = 12




        for i,val in enumerate(arcs_yyyyddd):

            if pd.to_datetime(val[:8],format="%Y.%j").month == m_num and pd.to_datetime(val[:8],format="%Y.%j").year == y_num:
                epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
                arc_length.append(epoch_delta.total_seconds()/3600)

                if val[8:] == 'A' or val[8:] == 'B':
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 scaleparameter_times.append(dict_sftime[arcs_yyyyddd[i]])
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime( epochstop[i] ),
                                                              freq=str(scale_cadence)+"h")
                    
                    if epochstop[i] in scaletimes:  ## dont add repeats to the final epoch time
                        ### Append the epoch end time to the end
                        add_epochend = scaletimes.values.astype(np.int64) // 10 ** 9
                                        #np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      #pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)

                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])


                    else:
                        ### Append the epoch end time to the end
                        add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])

                    pass
                else:

                    ### Use Arc text file times
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 print(f"epoch_top {epochstop[i]}")
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime(scalestop ),
                                                              freq=str(scale_cadence)+"h")

                    ### Append the epoch end time to the end
                    add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                  pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                    scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                        format ='%y%m%d%H%M%S.%f' )
                                     for ts in add_epochend ])
for i,val in enumerate(input_arcs):
    print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
#     if val[8:] == 'A' or val[8:] == 'B':
    print(f" {'':10}  , {scaleparameter_times[i]}")
    print("") 



In [ ]:
def setsettings(den, y_num, month, run_dict, scaling_cadence, file_raw_ICs, file_g2b, input_arcs, input_epochstart, input_epochstop, scaleparameter_times):
    
    settings_ice= {# Basic input settings
                 'satellite'      : {'input': 'icesat2'},
                 'den_model'      : {'input': den},
                 'run_type'       : {'input': 'DataReduction_PCE'},
                 'run_specifier'  : {'input': '_'+str(y_num)+month},
                 'cd_model'       : {'input': 'DRIA'},
                 'file_string'    : {'input': 'DRIAscaled_'},
                 'model_data_path' : {'input': run_dict[str(y_num)+month+den]['model_path']},
                 # Force Model settings
                  'cd_type'               : {'input':"Fixed_CD"},
                  'cd_value'              : {'input':1.0 },
                  'scaling_factor'        : {'input':True},
                  'hours_between_cd_adj'  : {'input':scaling_cadence},
                  #### Comment for unadjusted run:
                  'cd_adjustment_boolean' : {'input':True },
                #### DRIA CD Model Parameters
                'cd_model_params' : {'input':{ 
                        'MS'     : '26.980D0'   ,  #!  molar mass for each panel (g/mol)
                        'TW'     : '300.0D0'    ,  #!  temperature of panels  (K)
                           ###  Alpha is b/w 0 and 1
                        'ALPHA'  : '0.890D0'    ,  #!  accomodation coefficient
                        'KL'     : '0.0D0'    ,    #!  langmuir parameter
                        'FRACOX' : '1.0D0'   ,     #!  fraction of surface covered by atomic oxygen
                   }},
                  #### ---------------------------------------
                 # Run
                  'arc_type'       : {'input':'Nominal30hr_and_AB'},      
                  'step'           : {'input': 10},
                  'orbfil_step'    : {'input': 120.},
                  'which_ICfile'   : {'input':file_raw_ICs},
                  'which_g2bfile'  : {'input':file_g2b},
                    #
                  'arc'            : {'input': input_arcs},
                  'epoch_start'    : {'input': input_epochstart},
                  'epoch_stop'     : {'input': input_epochstop},  
            'scaleparameter_times' : {'input': scaleparameter_times},  
                   #                                
                  'global_options' : {'input':'pso_2018'},
                 # Request read on raw outputs
                  'request_data'   : {'input': ['Trajectory_orbfil', 
                                               'Density', 
                                               'Residuals_summary',
                                               'DragFile',
                                               'AdjustedParams'
                                               ]},
              #end dict
              }
    return(settings_ice)


In [ ]:
%load_ext autoreload
%autoreload 2
from gc import collect as gc_collect
import pickle 
import os
from datetime import datetime,timedelta


g2b_path = "/data/SatDragModelValidation/data/inputs/sat_icesat2/g2b/"

dir_save    =  '/data/SatDragModelValidation/data/outputs_clean/'\
             + 'icesat2/FinalDatasets_SWJ2025/'

obj = {}

for y_num in [2018, 2019 ]: #

    for imonth,month in enumerate(month_list):
        if month=='jan':  m_num = 1
        if month=='feb':  m_num = 2
        if month=='mar':  m_num = 3
        if month=='apr':  m_num = 4
        if month=='may':  m_num = 5
        if month=='jun':  m_num = 6
        if month=='jul':  m_num = 7
        if month=='aug':  m_num = 8
        if month=='sep':  m_num = 9
        if month=='oct':  m_num = 10
        if month=='nov':  m_num = 11
        if month=='dec':  m_num = 12
            
        if y_num==2018 and m_num<10:
            print(f'Skipping {y_num}.{m_num}')
            continue
        if y_num==2019 and m_num>4:
            print(f'Skipping {y_num}.{m_num}')
            continue
            
            
        file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_{y_num}_{m_num:02d}.txt"
        file_g2b     = f"pce_icesat2_pso_{y_num}_{m_num:02d}"
    #     file_raw_ICs = f"{g2b_path}ICESat2_RawEphem_20190331_20190503.txt"
    #     file_g2b     = f"pce_icesat2_pso_20190331_20190503"

        input_arcs           = []
        input_epochstart     = []
        input_epochstop      = []
        arc_length           = []
        scaleparameter_times = []

        for i,val in enumerate(arcs_yyyyddd):


            if pd.to_datetime(val[:8],format="%Y.%j").month == m_num and pd.to_datetime(val[:8],format="%Y.%j").year == y_num:
                epoch_delta  =  pd.to_timedelta(pd.to_datetime(epochstop[i]) - pd.to_datetime(epochstart[i]), 'hours')
                arc_length.append(epoch_delta.total_seconds()/3600)

                if val[8:] == 'A' or val[8:] == 'B':
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 scaleparameter_times.append(dict_sftime[arcs_yyyyddd[i]])
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i][:-1],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime( epochstop[i] ),
                                                              freq=str(scale_cadence)+"h")
                
                    if epochstop[i] in scaletimes:  ## dont add repeats to the final epoch time
                        ### Append the epoch end time to the end
                        add_epochend = scaletimes.values.astype(np.int64) // 10 ** 9
                                        #np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      #pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)

                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend  ])
                    else:
                        ### Append the epoch end time to the end
                        add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                      pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                        scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                            format ='%y%m%d%H%M%S.%f' )
                                         for ts in add_epochend ])
                else:

                    ### Use Arc text file times
                    input_arcs.append(      arcs_yyyyddd[i])
                    input_epochstart.append(epochstart[i])
                    input_epochstop.append( epochstop[i])
    #                 print(f"epoch_top {epochstop[i]}")
                    scalestart = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j"))
                    scalestop  = str(pd.to_datetime(arcs_yyyyddd[i],format="%Y.%j") + pd.to_timedelta(24,'h'))
                    scaletimes = pd.date_range(start = pd.to_datetime(scalestart),
                                                              end   = pd.to_datetime(scalestop ),
                                                              freq=str(scale_cadence)+"h")

                    ### Append the epoch end time to the end
                    add_epochend = np.append(scaletimes.values.astype(np.int64) // 10 ** 9,
                                  pd.to_datetime(pd.Series(epochstop[i])).values.astype(np.int64) // 10 ** 9)


                    scaleparameter_times.append([pd.to_datetime(datetime.strftime(datetime.fromtimestamp(ts), '%y%m%d%H%M%S.%f'),
                        format ='%y%m%d%H%M%S.%f' )
                                     for ts in add_epochend ])

        for i,val in enumerate(input_arcs):
            print(f" {input_arcs[i]:10}  , {input_epochstart[i]:20},  {input_epochstop[i]:20}, {arc_length[i]}")
            if val[8:] == 'A' or val[8:] == 'B':
                print(f" {'':10}  , {scaleparameter_times[i]}")
                print("")

            else:
        #         print(f" {'':10}  , {scaleparameter_times[i]}")
                print("")     
    
#         for i,den in enumerate(run_list):
#             if month in ['jan','feb', 'mar','apr'] :
#                 print(f"already ran {month} for {den}")

#             elif month == :# and den in ['msis2', 'jb2008']:
#                 print(f"already ran {month} for {den}")

#             elif month ==  and den in ['msis2', 'jb2008','dtm2020_o']:
#                 print(f"already ran {month} for {den}")
#             else:
                
#                 settings_ice = setsettings(den, y_num, month, run_dict, scaling_cadence, file_raw_ICs, file_g2b, 
#                                              input_arcs, 
#                                              input_epochstart, 
#                                              input_epochstop, 
#                                              scaleparameter_times)

#                 sat = Pygeodyn(settings_ice, use_file=False)
#                 sat.run_arcs()

        for i,den in enumerate(run_list):
            settings_ice = setsettings(den,y_num, month, run_dict, scaling_cadence, file_raw_ICs, file_g2b, 
                                         input_arcs, 
                                         input_epochstart, 
                                         input_epochstop, 
                                         scaleparameter_times)
            sat = Pygeodyn(settings_ice, use_file=False)
#             obj[str(y_num)+month+den] =  sat.getData_BigData_lowmemory(orbit_propagation=False)
#             obj[str(y_num)+month+den] = vars(obj[str(y_num)+month+den])
#             gc_collect()


            pickleName = f'_{y_num}.{month}_DRIA_scale{scale_cadence}.pkl'

            pickle_file = dir_save+den+pickleName
            if not os.path.exists(pickle_file):
                print('Must create pickle file...')
                print('   ',  pickle_file)
                print('   ', 'Reading Geodyn Data')

                ### Load the data into an object
                sat = Pygeodyn(settings_ice, use_file=False)
                obj = sat.getData_BigData_lowmemory(orbit_propagation=False)
                gc_collect()

                #### Pickle the object to save it
                print('   ', 'Saving pickle')
                filehandler = open(pickle_file, 'wb') 
                pickle.dump(vars(obj), filehandler)
                filehandler.close()
                obj = 0
                print('   ', 'Saved pickle')

    for i,model in enumerate(run_list):
#         print('RUNLIST',model)
        for imonth,month in enumerate(month_list):
            
            print('y_num month',y_num, month)
            if y_num==2018 and month not in ['oct', 'nov', 'dec']:
                print(f'Skipping {y_num}.{month}')
                
            elif y_num==2019 and month not in ['jan', 'feb', 'mar', 'apr']:
                print(f'Skipping {y_num}.{month}')
                
            else:
                pickleName = f'_{y_num}.{month}_DRIA_scale{scale_cadence}.pkl'

                ### Load the data if the pickles exist
                print()
                print()
                gc_collect()

                pickle_file = dir_save+model+pickleName

                obj[str(y_num)+month+model] = pd.read_pickle(pickle_file)
        #         filehandler = open(pickle_file, 'rb') 
        #         obj[month+model] = pickle.load(filehandler)
        #         filehandler.close()
                print('Loaded data from pickle... ',y_num, month, model)

### Save space if doing density retrieval
for model in run_dict.keys():
    print(model)
    del obj[model]['OrbitResids']
    del obj[model]['Trajectory_orbfil']

gc_collect()

In [ ]:
obj.keys()

In [ ]:
for model in run_dict.keys():
    print(model)

In [ ]:
run_dict

In [ ]:
# file_raw_ICs

In [ ]:
# arcs_yyyyddd

In [ ]:
# input_arcs

In [ ]:
# pd.to_datetime(val[:8],format="%Y.%j").year

In [ ]:
# input_arcs

In [ ]:
def plot_cleanformat_axes(fig, font_dict):
    rownum, colnum = fig._get_subplot_rows_columns()

    for i in rownum:
        if len(rownum)==1:
            L_ticklabel = True
        else:
            if i < len(rownum):
                L_ticklabel = False
            else:
                L_ticklabel = True
        fig.update_xaxes(### LINE at axis border
                          showline=True,
                          showticklabels=L_ticklabel,
    #                       tickformat= '%m/%d',
                          linecolor='black',
                          linewidth=1,
                         ### Major ticks
                          ticks='inside',
                          tickfont=font_dict,
                          mirror=True,
    #                       tickwidth=2,
    #                       ticklen=9,
                          tickcolor='grey',
    #                       tick0="2018-11-9" ,
    #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                          #### Minor Ticks
                           minor=dict(
                             dtick=86400000.0, # milliseconds in a day
                             tickwidth=1,
                             ticklen=4,
                             tickcolor='grey',
                             ticks='inside'),
                          ### GRID
                           gridcolor='gainsboro',
                           gridwidth=1,
                           layer='above traces',
                           tickangle=0,
                           row=i, col=1)
        fig.update_yaxes(showline=True,      # add line at x=0
                             showticklabels=True,
                             linecolor='black',  # line color
                             linewidth=1,        # line size
                         ticks='inside',     # ticks outside axis
                         tickfont=font_dict, # tick label font
                         mirror='allticks',  # add ticks to top/right axes
                         tickwidth=1,      # tick width
                         tickcolor='black',  # tick color
                         gridcolor='gainsboro',
                         gridwidth=1,
                         layer='above traces',
                         row=i, col=1)
    return(fig)

from pygeodyn.pygeodyn_plot_scalingfactors import *

coldict = {}
coldict['msis2']     = "#2ca02c"  # 'tab:green'
coldict['dtm2020_o'] = "#d62728"  # 'tab:red'
coldict['jb2008']    = "orange"   
coldict['hasdm_oc']  = "#1f77b4"     


In [ ]:
  
satid = 1807001
wgts = {}

for model in run_dict.keys():
    wgts[model] = {}
    ScalingFactors  = []
    ScalingFactor_times = []

    for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
        epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
        hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
        frachours =(hrs/24)
        #
        if len(arc) == 9:
            maneuv_indicator = arc[8]
        else:
            maneuv_indicator = ''
        arc_type = obj[model]['global_params']['prms']['arc_type']
        if arc_type == "Nominal30hr_and_AB":
            arc_name =arc[:8]+ maneuv_indicator
            arc_name_DOY = arc[:8]
        else:
            arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        ### Collect the weights for the ensemble average
        inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
        wgts[model][arc_name_DOY] = inv_rms#/sum_wgts

        iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
        for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):
            
            numSFs = len(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'])
            
            if iit == 0 or iit==numSFs-1:   # remove the first and last Scaling Factor to account for edge effects
                pass
            else:
                CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
                APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
                ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
                ScalingFactor_times.append(itime)
    run_dict[model]['ScalingFactor_times'] = ScalingFactor_times
    run_dict[model]['ScalingFactors']      = ScalingFactors
    run_dict[model]['Weight'] = wgts[model]
    

    
# ###### SCALE THE DENSITIES:
# models_dens = {}
# for monthmodel in run_dict.keys():
#     print(f"---Making continuous scaled rho for {monthmodel}")
#     models_dens[monthmodel] = get_continuous_scaled_densities(obj, run_dict, monthmodel, scale_cadence)

#     del obj[monthmodel]
#     gc_collect()

# del obj
# gc_collect()

# # Retrieve scaled ensemble weighted average
# #     'Rho_x' denotes the ensemble weighted avg
# print(f"")
# print(f"--- Processing Rho_x")
# yearmonth_list = ['2018oct',
#               '2018nov',
#               '2018dec',
#               '2019jan',
#               '2019feb',
#               '2019mar',
#               '2019apr']
# models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, yearmonth_list, run_list)




In [ ]:
%load_ext autoreload
%autoreload 2


def PLOT_retrievedRHO(fig, obj_m1,  model_dict, model_name):
        
    obj_m1=obj_m1[model_name]
#     print(model_name)
    model_m1 = model_name[7:]
#obj_m1['global_params']['prms']['den_model']

    fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
                               y=model_dict['ScalingFactors'],
                               name= model,
                               mode='markers',
                               opacity=1,
#                                    marker=dict(color=coldict[model_m1], 
#                                                size=5,
#                                                symbol='line-ew',
#                                                line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
                                   marker=dict(color=coldict[model_m1], size=5),
                               line = dict(shape = 'hvh',dash ='dot', color = coldict[model_m1], width=1),
                               showlegend=False),
                               secondary_y=False,row=1, col=1)

    if model_m1 == 'dtm2020_o':
        
    
        for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
            epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
            hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
            frachours =(hrs/24)
            #
            if len(arc) == 9:
                maneuv_indicator = arc[8]
            else:
                maneuv_indicator = ''
            arc_type = obj_m1['global_params']['prms']['arc_type']
            if arc_type == "Nominal30hr_and_AB":
                arc_name =arc[:8]+ maneuv_indicator
            else:
                arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator

            drag_data        = obj_m1['DragFile'][arc_name]
            drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
            den_df           = obj_m1['Density' ][arc_name]

    #         print(arc[:8])
            ### Cut off the extra time for the regular days
            start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
            end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

    #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")

            ### SELECT ONLY THE MIDDLE 24-HOURS
            drag_data = drag_data.query( \
                    f"{start_arc.year}"         \
                   +f"{start_arc.month:02d}"    \
                   +f"{start_arc.day:02d}"      \
                   +f"{start_arc.hour:02d}"     \
                   +f"{start_arc.minute:02d}"   \
                   +f"{start_arc.second:02d}"   \
                   +f" <= Date < "                     \
                   +f"{end_arc.year}"       \
                   +f"{end_arc.month:02d}"  \
                   +f"{end_arc.day:02d}"    \
                   +f"{end_arc.hour:02d}"   \
                   +f"{end_arc.minute:02d}" \
                   +f"{end_arc.second:02d}" \
                )

            indplot = 4
            fig.add_trace(go.Scattergl(x=drag_data['Date'],
                                   y=drag_data['TOTAREA'],
                                 mode='markers',
                                 opacity=1,
                                 marker=dict(color= 'gray',size=2,),
                                 showlegend=False,),
                                 secondary_y=False,
                                 row=2, col=1,
                             )

#             (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#             (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    

#             fig.add_trace(go.Scattergl(x=time_avg,
#                                      y=avg_cd*avg_area,
#     #                                  y=avg_cd,
#                                mode='lines',
#                                opacity=1,
#                                    line = dict(dash ='solid', color = 'black', width=3),
#                                showlegend=False), row=2, col=1)
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                    y=drag_data['TOTAREA'][::indplot],
#                                  mode='markers',
#                                  opacity=0.2,
#                                  marker=dict(color= 'gray',size=2,),
#                                  showlegend=False,),
#                                  secondary_y=False,
#                                  row=2, col=1,
#                              )

            (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    

            fig.add_trace(go.Scattergl(x=time_avg,
                                     y=avg_area,
    #                                  y=avg_cd,
                               mode='lines',
                               opacity=1,
                                   line = dict(dash ='solid', color = 'black', width=3),
                               showlegend=False), row=3, col=1)
        else:
            pass
    
             
    return(fig)







fig  = make_subplots(
    rows=3, cols=1,
    vertical_spacing = 0.05,
    shared_xaxes=True)



# run_dict = { }
# rms_total_return = {}

for model in run_dict.keys():
    print(model)
#     for imonth,month in enumerate(month_list):

#     fig = PLOT_retrievedRHO(fig, obj[model],   models_dens[model], run_dict[model], model)
    fig = PLOT_retrievedRHO(fig, obj,    run_dict[model], model)

    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = False
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    fig.update_yaxes(title_text="Scaling Factor", 
                     exponentformat= 'power',row=1, col=1)

#     yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range

#     fig.update_yaxes(title_text="Scaled Density", 
#                      type="log", 
# #                       range=yaxis_range,
#                      exponentformat= 'power',row=2, col=1)
    fig.update_yaxes(title_text="A", 
#                       type="log", 
#                       range=yaxis_range,
                     exponentformat= 'power',row=2, col=1)
    fig.update_yaxes(title_text="Area", 
#                       type="log", 
#                       range=yaxis_range,
                     exponentformat= 'power',row=3, col=1)
    
    
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# a='input'
# s=settings_icesat2

fig.update_layout(#title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}",
#                     title=f"BWDRAG, 3-hr Scale from CD=2.5 ",
                  autosize=False,    width=900,    height=500,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
fig.show(config= dict({
                'displayModeBar': False,
                'responsive': True,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                }),
#          renderer='jpg',
)


# pio.write_image(fig, 'ScalingFactors_ScaledRho.jpg', scale=5)




In [ ]:
# # Retrieve scaled ensemble weighted average
# #     'Rho_x' denotes the ensemble weighted avg
# yearmonth_list = ['2018oct',
#               '2018nov',
#               '2018dec',
#               '2019jan',
#               '2019feb',
#               '2019mar',
#               '2019apr']
# print(f"--- Processing Rho_x")
# models_dens =  calc_rho_ScaledEnsembleWgtAvg(models_dens, run_dict, yearmonth_list, run_list)



In [ ]:
# models_dens.keys()

In [ ]:
# for ii,arc in enumerate(obj[model]['global_params']['arc_input']):
#     print()
#     epochstart = obj[model]['global_params']['prms']['epoch_start'][ii]
#     hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#     frachours =(hrs/24)
#     #
#     if len(arc) == 9:
#         maneuv_indicator = arc[8]
#     else:
#         maneuv_indicator = ''
#     arc_type = obj[model]['global_params']['prms']['arc_type']
#     if arc_type == "Nominal30hr_and_AB":
#         arc_name =arc[:8]+ maneuv_indicator
#         arc_name_DOY = arc[:8]
#     else:
#         arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
#     ### Collect the weights for the ensemble average
#     inv_rms          = 1/obj[model]['Statistics'][arc_name]['T_RMS'].values[0]
#     wgts[model][arc_name_DOY] = inv_rms#/sum_wgts

#     iters = int(obj[model]['run_parameters'+arc_name]['total_iterations']) 
#     for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):

#         numSFs = len(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'])

#         if iit == 0 or iit==numSFs-1:   # remove the first and last Scaling Factor to account for edge effects
#             pass
#         else:
#             CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
#             APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
#             ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
#             ScalingFactor_times.append(itime)
#             print(f"{itime} : {CURRENT_VALUE/APRIORI_VALUE}")


In [ ]:
# for iit, itime in enumerate(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'].keys()):

#     numSFs = len(obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'])

# #         if iit == 0 or iit==numSFs-1:   # remove the first and last Scaling Factor to account for edge effects
# #             pass
# #         else:
#     CURRENT_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['CURRENT_VALUE']
#     APRIORI_VALUE = obj[model]['AdjustedParams'][arc_name][iters][satid]['0CD'][itime]['APRIORI_VALUE']
# #     ScalingFactors.append(CURRENT_VALUE/APRIORI_VALUE)
# #     ScalingFactor_times.append(itime)
    
#     print(f"{itime} : {CURRENT_VALUE/APRIORI_VALUE}")

In [ ]:
import linecache
import pandas as pd
import numpy as np

path_to_flux = "/data/SatDragModelValidation/src/geodyn_code/support/geodyn_support/make_tables/"


dateS   = []
dateAP  = []
dateKP8 = []

fluxS   = []
fluxAP  = []
fluxKP8 = []

switch = 0
with open(path_to_flux+"new_master", 'r') as f:
    for line_no, string in enumerate(f):
#         string = linecache.getline("new_master",line+1) #

        if 'MASTER' in string:        
            switch = 'skip'
            continue
            
        ### Skip the title lines and any dates before Y2K
        if 'FLUXS' in string:        
            switch = 'readfluxS'
            print(switch)
            continue

        if 'FLUXAP' in string:        
            switch = 'readfluxAP'
            print(switch)
            continue
        if 'FLUXKP8' in string:        
            switch = 'readfluxKP8'
            print(switch)
            continue

        if switch =='skip':
            continue
            
        if int(string[:2]) >= 58:
            continue
        else:
            
            if int(string[:2]) < 18 or int(string[:2]) > 19:
                continue
                
            if switch == 'readfluxS':
                ### Load the Solar Flux values after 2000. The date leading each line represents
                ### the first flux value in that line.  The following n values are that day + n days.
                ###  All Solar Flux values are mulitplied by 10 for some reason...

                for n in range(0,12):
                    dateS.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxS.append(int(string[11:15]) / 10)
                fluxS.append(int(string[16:20]) / 10)
                fluxS.append(int(string[21:25]) / 10)
                fluxS.append(int(string[26:30]) / 10)
                fluxS.append(int(string[31:35]) / 10)
                fluxS.append(int(string[36:40]) / 10)
                fluxS.append(int(string[41:45]) / 10)
                fluxS.append(int(string[46:50]) / 10)
                fluxS.append(int(string[51:55]) / 10)
                fluxS.append(int(string[56:60]) / 10)
                fluxS.append(int(string[61:65]) / 10)
                fluxS.append(int(string[66:70]) / 10)

            if switch == 'readfluxAP':
                ### Load the AP values after 2000. These are formatted the same as the FLUXS section
                for n in range(0,12):
                    dateAP.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(1,'d') )
                fluxAP.append(int(string[11:15]) / 10)
                fluxAP.append(int(string[16:20]) / 10)
                fluxAP.append(int(string[21:25]) / 10)
                fluxAP.append(int(string[26:30]) / 10)
                fluxAP.append(int(string[31:35]) / 10)
                fluxAP.append(int(string[36:40]) / 10)
                fluxAP.append(int(string[41:45]) / 10)
                fluxAP.append(int(string[46:50]) / 10)
                fluxAP.append(int(string[51:55]) / 10)
                fluxAP.append(int(string[56:60]) / 10)
                fluxAP.append(int(string[61:65]) / 10)
                fluxAP.append(int(string[66:70]) / 10)

            if switch == 'readfluxKP8':
                ### Load the KP values after 2000. These are formatted such that each line is a day
                ### and each nth value in the line is n*3hrs for the day
                for n in range(0,8):
                    dateKP8.append(pd.to_datetime(string[:6],format='%y%m%d') + n*pd.to_timedelta(3,'h') )
                fluxKP8.append(int(string[7:11])  / 100)
                fluxKP8.append(int(string[12:16]) / 100)
                fluxKP8.append(int(string[17:21]) / 100)
                fluxKP8.append(int(string[22:26]) / 100)
                fluxKP8.append(int(string[27:31]) / 100)
                fluxKP8.append(int(string[32:36]) / 100)
                fluxKP8.append(int(string[37:41]) / 100)
                fluxKP8.append(int(string[42:46]) / 100)

fluxS = np.array(fluxS)
fluxAP = np.array(fluxAP)
fluxKP8 = np.array(fluxKP8)
                
fluxS   = np.where(fluxS  == 0., np.nan, fluxS  )
fluxAP  = np.where(fluxAP ==0., np.nan, fluxAP )
fluxKP8 = np.where(fluxKP8==0., np.nan, fluxKP8)



In [ ]:
dateS[300]

In [ ]:
%load_ext autoreload
%autoreload 2


def PLOT_resids(fig,  den_dict, model_dict, _model_name_):
    
    SHOW_alldata = True
    
    model_m1 = _model_name_[7:]
    print('model_m1', model_m1)

    fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
                           y=model_dict['ScalingFactors'],
                           name= model,
                           mode='markers',
                           opacity=1,
                           marker=dict(color=coldict[model_m1], size=5),
#                                marker=dict(color=coldict[model_m1], 
#                                            size=6,
#                                            symbol='line-ew',
#                                            line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
#                            line = dict(shape = 'hvh',dash ='dash', color = coldict[model_m1], width=1),
                           showlegend=False),
                           secondary_y=False,row=2, col=1)

    
    
    time_avg,dscale_avg = orbit_avg_generic(den_dict['dates'], den_dict['denscaled'], den_dict['lat'])
    ## -----------------------------------------------------------------------------------------------------
    ##     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=dscale_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color=coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=4, col=1)
#     fig.add_trace(go.Scattergl(x=den_dict['dates'],
#                                y=den_dict['denscaled'],
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=coldict[model_m1],size=2),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=3, col=1)

    
    
    
    
    time_avg,d_avg = orbit_avg_generic(den_dict['dates'], den_dict['dens'], den_dict['lat'])
    ### -----------------------------------------------------------------------------------------------------
    ###     Orbit Averaged Density
#     fig.add_trace(go.Scattergl(x=den_dict['dates'], #time_avg,
#                                y=den_dict['dens'], #d_avg,
#                                ### name= model_m1,
#                                mode='markers+lines',
#                                opacity=1,
#                                    marker=dict(color=coldict[model_m1],size=1),
#                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                                showlegend=False), row=2, col=1)

    ###     Orbit Averaged Density
    fig.add_trace(go.Scattergl(x=time_avg,
                               y=d_avg,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                                   marker=dict(color= coldict[model_m1],size=2),
                                   line = dict(dash ='solid', color = coldict[model_m1], width=2),
                               showlegend=False), row=3, col=1)
    
    
#                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
#                 fig.add_trace(go.Scattergl(x=time_avg,
#                                            y=area_avg_rolling,
#     #                                          name= plot_name+arc,
#                                              mode='markers+lines',
#                                              opacity=1,
#                                              marker=dict(color=col,size=2,),
#                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
#                                              showlegend=False, ),
#                                              secondary_y=False,
#                                              row=3, col=1,
#                                          )

    return(fig)


fig  = make_subplots(
    rows=4, cols=1,
    vertical_spacing = 0.05,
    shared_xaxes=True)


fig = make_subplots(rows=4, cols=1, row_heights=[0.1, 0.3, 0.3, 0.3],#],
                    specs=[[{"secondary_y": True}],
                           [{"secondary_y": False}],
                           [{"secondary_y": False}],
                           [{"secondary_y": False}]],
                           shared_xaxes=True,
                           vertical_spacing=0.02)




fig.add_trace(go.Scattergl(x=dateS,
                           y=fluxS,
                           name= 'F107d_1AU',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh',dash='dash', color = 'blue', width=2),
                           showlegend=False),
                           secondary_y=True,row=1, col=1)

fig.add_trace(go.Scattergl(x=dateKP8,
                           y=fluxKP8,
                           name= 'Kp',
                           mode='lines',
                           opacity=1,
                           line = dict(shape = 'hvh', color = 'black', width=2),
                           showlegend=False),
                           secondary_y=False,row=1, col=1) 

fig.update_yaxes(title_text="Kp", 
                 exponentformat= 'power',
#                  range=[0,7],
                 secondary_y=False,
                 row=1, col=1)
fig.update_yaxes(title_text="F10.7", 
                 exponentformat= 'power',
                 range=[35,170],
                 secondary_y=True,
                 tickfont=dict(color="blue"),
                 titlefont=dict(color="blue"),
                 row=1, col=1)

for model in run_dict.keys():
#     for imonth,month in enumerate(month_list):
    print('model', model)

    fig = PLOT_resids(fig, models_dens[model], run_dict[model], model)

    
#######################################################
font_dict=dict(family='Arial',size=11,color='black')
## automate the specification of the axes for subplots
rownum, colnum = fig._get_subplot_rows_columns()
for i in rownum:
    if len(rownum)==1:
        L_ticklabel = True
    else:
        if i < len(rownum):
            L_ticklabel = True
        else:
            L_ticklabel = True
    fig.update_xaxes(### LINE at axis border
                      showline=True,
                      showticklabels=L_ticklabel,
#                       tickformat= '%m/%d',
                      linecolor='black',
                      linewidth=1,
                     ### Major ticks
                      ticks='inside',
                      tickfont=font_dict,
                      mirror=True,
#                       tickwidth=2,
#                       ticklen=9,
                      tickcolor='grey',
#                       tick0="2018-11-9" ,
#                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
                      #### Minor Ticks
                       minor=dict(
                         dtick=86400000.0, # milliseconds in a day
                         tickwidth=1,
                         ticklen=4,
                         tickcolor='grey',
                         ticks='inside'),
                      ### GRID
                       gridcolor='gainsboro',
                       gridwidth=1,
                       layer='above traces',
                       tickangle=0,
                       row=i, col=1)
    fig.update_yaxes(showline=True,      # add line at x=0
                         showticklabels=True,
                         linecolor='black',  # line color
                         linewidth=1,        # line size
                     ticks='inside',     # ticks outside axis
                     tickfont=font_dict, # tick label font
                     mirror='allticks',  # add ticks to top/right axes
                     tickwidth=1,      # tick width
                     tickcolor='black',  # tick color
                     gridcolor='gainsboro',
                     gridwidth=1,
                     layer='above traces',
                     row=i, col=1)

    fig.update_yaxes(title_text="Scaling Factor", 
                     exponentformat= 'power',row=2, col=1)

    fig.update_yaxes(title_text="Density", 
                     type="log", 
                     exponentformat= 'power',row=3, col=1)
    fig.update_yaxes(title_text="Scaled Density", 
                      type="log", 
                     exponentformat= 'power',row=4, col=1)

for month in yearmonth_list:

    time_avg,Rho_x = orbit_avg_generic(models_dens[month+'Rho_x']['date'],
                                            models_dens[month+'Rho_x']['Rho_x'],
                                            models_dens[month+'Rho_x']['lat'])

    time_avg,Rho_std = orbit_avg_generic(models_dens[month+'Rho_x']['date'],
                                            models_dens[month+'Rho_x']['Rho_std'],
                                            models_dens[month+'Rho_x']['lat'])


    fig.add_trace(go.Scattergl(x=time_avg,
                               y=Rho_x,
                               ### name= model_m1,
                               mode='markers+lines',
                               opacity=1,
                               marker=dict(color='black',size=5),
                                   line = dict( dash ='solid', color = 'black', width=4),
                               showlegend=False), row=4, col=1)
    ## ERRROR BARS
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=np.array(Rho_x)\
#                                    +np.array(Rho_std),
#                                ### name= model_m1,
#                                mode='lines',
#                                opacity=.8,
#                                    line = dict( dash ='dash', color = 'grey', width=3),
#                                showlegend=False), row=4, col=1)
#     fig.add_trace(go.Scattergl(x=time_avg,
#                                y=np.array(Rho_x)\
#                                   -np.array(Rho_std),
#                                ### name= model_m1,
#                                mode='lines',
#                                opacity=.8,
#                                    line = dict( dash ='dash', color = 'grey', width=3),
#                                showlegend=False), row=4, col=1)

    
fig.update_xaxes(range=[pd.to_datetime('2018-10-14 00:00:00'), pd.to_datetime('2019-05-01 00:00:00')])

    
a='input'
s=settings_ice
fig.update_layout(title=f" ICESat-2 {s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale", 
                  autosize=False,    width=850,    height=900,
                  legend= {'itemsizing': 'trace'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config=config)
fig.show(config= dict({
                'displayModeBar': False,
                'responsive': True,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                }),
#          renderer='jpg',
)




In [ ]:
# obj['mardtm2020_o']['OrbitResids']['2019.078'].keys()


In [ ]:
# obj['mardtm2020_o']['OrbitResids']['2019.078']['resids'].keys()


### NTW residuals

In [ ]:

# fig  = make_subplots(
#     rows=3, cols=1,
#     vertical_spacing = 0.05,
#     shared_xaxes=True)

# for ii,arc in enumerate(obj['mardtm2020_o']['global_params']['arc_input']):
#     epochstart = obj['mardtm2020_o']['global_params']['prms']['epoch_start'][ii]
#     hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#     frachours =(hrs/24)
#     #
#     if len(arc) == 9:
#         maneuv_indicator = arc[8]
#     else:
#         maneuv_indicator = ''
#     arc_type = obj['mardtm2020_o']['global_params']['prms']['arc_type']
#     if arc_type == "Nominal30hr_and_AB":
#         arc_name =arc[:8]+ maneuv_indicator
#     else:
#         arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['N'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=1, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['N'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=1, col=1)
    
#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['T'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=2, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['T'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=2, col=1)

#     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['Date'],
#                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['resids']['W'],
#                                mode='markers+lines',
#                                opacity=1,
#                                marker=dict(color='blue', size=3),
#                                    showlegend=False),
#                                    secondary_y=False,row=3, col=1)
# #     fig.add_trace(go.Scattergl(x=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['Date'],
# #                                y=obj['mardtm2020_o']['OrbitResids'][arc_name]['data_orbfil']['W'],
# #                            mode='markers',
# #                            opacity=1,
# #                                marker=dict(color='red', size=3),
# #                                showlegend=False),
# #                                secondary_y=False,row=3, col=1)  
    
    
    
    
# fig.show(config= dict({
#                 'displayModeBar': False,
#                 'responsive': False,
#                 'staticPlot': True,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }),
#         )



### SF and CD*A

In [ ]:
# %load_ext autoreload
# %autoreload 2


# def PLOT_retrievedRHO(fig, obj_m1, den_dict, model_dict, model_name):
        
#     obj_m1=obj_m1[model_name]
# #     print(model_name)
#     model_m1 = model_name[3:]#obj_m1['global_params']['prms']['den_model']

#     fig.add_trace(go.Scattergl(x=pd.to_datetime(model_dict['ScalingFactor_times'])-pd.to_timedelta(scale_cadence/2, 'h'),
#                                y=model_dict['ScalingFactors'],
#                                name= model,
#                                mode='markers',
#                                opacity=1,
# #                                    marker=dict(color=coldict[model_m1], 
# #                                                size=5,
# #                                                symbol='line-ew',
# #                                                line = dict(color = coldict[model_m1], width=3)), #symbol='diamond-wide'),
#                                    marker=dict(color=coldict[model_m1], size=5),
#                                line = dict(shape = 'hvh',dash ='dot', color = coldict[model_m1], width=1),
#                                showlegend=False),
#                                secondary_y=False,row=1, col=1)

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        
#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )

#         indplot = 4
#         fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
#                              mode='markers',
#                              opacity=0.3,
#                              marker=dict(color= coldict[model_m1],size=2,),
#                              showlegend=False,),
#                              secondary_y=False,
#                              row=2, col=1,
#                          )
    
#         (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    

#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd*avg_area,
# #                                  y=avg_cd,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=2, col=1)
    
    
             
#     return(fig)







# fig  = make_subplots(
#     rows=2, cols=1,
#     vertical_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
#     print(model)
# #     for imonth,month in enumerate(month_list):

# #     fig = PLOT_retrievedRHO(fig, obj[model],   models_dens[model], run_dict[model], model)
#     fig = PLOT_retrievedRHO(fig, obj,   models_dens[model], run_dict[model], model)


        

# # time_avg,Rho_x = orbit_avg_generic(models_dens['novRho_x']['date'],
# #                                         models_dens['novRho_x']['Rho_x'],
# #                                         models_dens['novRho_x']['lat'])

# # time_avg,Rho_std = orbit_avg_generic(models_dens['novRho_x']['date'],
# #                                         models_dens['novRho_x']['Rho_std'],
# #                                         models_dens['novRho_x']['lat'])


# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=Rho_x,
# #                            ### name= model_m1,
# #                            mode='markers+lines',
# #                            opacity=1,
# #                            marker=dict(color='black',size=5),
# #                                line = dict( dash ='solid', color = 'black', width=4),
# #                            showlegend=False), row=2, col=1)
# # ## ERRROR BARS
# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=np.array(Rho_x)\
# #                                +np.array(Rho_std),
# #                            ### name= model_m1,
# #                            mode='lines',
# #                            opacity=.8,
# #                                line = dict( dash ='dash', color = 'grey', width=3),
# #                            showlegend=False), row=2, col=1)
# # fig.add_trace(go.Scattergl(x=time_avg,
# #                            y=np.array(Rho_x)\
# #                               -np.array(Rho_std),
# #                            ### name= model_m1,
# #                            mode='lines',
# #                            opacity=.8,
# #                                line = dict( dash ='dash', color = 'grey', width=3),
# #                            showlegend=False), row=2, col=1)

    
    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

#     fig.update_yaxes(title_text="Scaling Factor", 
#                      exponentformat= 'power',row=1, col=1)

# #     yaxis_range = [-13.7 -.55 ,  -12.6+.25]# ] #full_fig.layout.yaxis2.range

# #     fig.update_yaxes(title_text="Scaled Density", 
# #                      type="log", 
# # #                       range=yaxis_range,
# #                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="CD*A", 
# #                       type="log", 
# #                       range=yaxis_range,
#                      exponentformat= 'power',row=2, col=1)
    
    
# fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),)

# # a='input'
# # s=settings_icesat2

# fig.update_layout(#title=f"{s['cd_model'][a]}, {s['hours_between_cd_adj'][a]}-hr Scale from CD={s['cd_value'][a]}",
# #                     title=f"BWDRAG, 3-hr Scale from CD=2.5 ",
#                   autosize=False,    width=900,    height=550,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title
# fig.show(config= dict({
#                 'displayModeBar': False,
#                 'responsive': True,
#                 'staticPlot': False,
#                 'displaylogo': False,
#                 'showTips': False,
#                 }),
# #          renderer='jpg',
# )


# # pio.write_image(fig, 'ScalingFactors_ScaledRho.jpg', scale=5)




### ful drag state

In [ ]:
# %load_ext autoreload
# %autoreload 2


# def PLOT__SATDRAG_STATE(fig, obj_m1, den_dict):
    
#     SHOW_alldata = True
    
#     model_m1 = obj_m1['global_params']['prms']['den_model']
#     if model!='hasdm_oc':
#         opac_val = 1
#     else:
#         opac_val = 1

#     for ii,arc in enumerate(obj_m1['global_params']['arc_input']):
#         epochstart = obj_m1['global_params']['prms']['epoch_start'][ii]
#         hrs = pd.to_datetime(epochstart, format='%Y-%m-%d %H:%M:%S').hour
#         frachours =(hrs/24)
#         #
#         if len(arc) == 9:
#             maneuv_indicator = arc[8]
#         else:
#             maneuv_indicator = ''
#         arc_type = obj_m1['global_params']['prms']['arc_type']
#         if arc_type == "Nominal30hr_and_AB":
#             arc_name =arc[:8]+ maneuv_indicator
#         else:
#             arc_name =arc[:8]+('%.3f'%frachours).lstrip('0')+ maneuv_indicator
        



#         drag_data        = obj_m1['DragFile'][arc_name]
#         drag_data["Lat"] = obj_m1['Density' ][arc_name]['Lat']
#         den_df           = obj_m1['Density' ][arc_name]
        
# #         print(arc[:8])
#         ### Cut off the extra time for the regular days
#         start_arc = pd.to_datetime(arc[:8], format='%Y.%j')
#         end_arc   = pd.to_datetime(arc[:8], format = '%Y.%j')+pd.to_timedelta(24, 'h')-pd.to_timedelta(1, 'm')

# #         print(f" {arc_name:10}|   , {start_arc},  {end_arc}")
        
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         drag_data = drag_data.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )
#         ### SELECT ONLY THE MIDDLE 24-HOURS
#         den_df = den_df.query( \
#                 f"{start_arc.year}"         \
#                +f"{start_arc.month:02d}"    \
#                +f"{start_arc.day:02d}"      \
#                +f"{start_arc.hour:02d}"     \
#                +f"{start_arc.minute:02d}"   \
#                +f"{start_arc.second:02d}"   \
#                +f" <= Date < "                     \
#                +f"{end_arc.year}"       \
#                +f"{end_arc.month:02d}"  \
#                +f"{end_arc.day:02d}"    \
#                +f"{end_arc.hour:02d}"   \
#                +f"{end_arc.minute:02d}" \
#                +f"{end_arc.second:02d}" \
#             )


#         indplot=3

# #         date_rms.append(pd.to_datetime(datetime.datetime(int(arc.split('.')[0]), 1, 1) + datetime.timedelta(int(arc.split('.')[1]))- datetime.timedelta(hours=12) ))
# #         rms_totals.append(obj_m1['Statistics'][arc]['T_RMS'].values[0])

# #         iters = obj_m1['run_parameters'+arc_name]['total_iterations']
# #         for itime in obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD']:
# #             date_scalefactor.append(itime)
# #             ScalingFactor.append(obj_m1['AdjustedParams'][arc_string][iters][satid]['0CD'][itime]['CURRENT_VALUE'])


#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=2, col=1,
#                                  )
#         (time_avg, avg_cd ) = orbit_avg_generic(drag_data['Date'],drag_data['CD'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=2, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=3, col=1,
#                                  )
#         (time_avg, avg_area) = orbit_avg_generic(drag_data['Date'],drag_data['TOTAREA'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=3, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['CD'][::indplot]*drag_data['TOTAREA'][::indplot],
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      showlegend=False,),
#                                      secondary_y=False,
#                                      row=4, col=1,
#                                  )
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg_cd*avg_area,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=4, col=1)

        
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                        y=drag_data['VELREL'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=5, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['VELREL'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=5, col=1)

#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=drag_data['Date'][::indplot],
#                                    y=drag_data['SpeedRatio'][::indplot],
# #                                          name= plot_name+arc,
#                                      mode='markers',
#                                      opacity=0.5,
#                                      marker=dict(color= coldict[model_m1],size=2,),
#                                      line = dict(shape='hvh', dash ='solid', color = coldict[model_m1], width=2),
#                                      showlegend=False, ),
#                                      secondary_y=False,
#                                      row=6, col=1,
#                                  )
#         (time_avg, avg ) = orbit_avg_generic(drag_data['Date'],drag_data['SpeedRatio'],drag_data['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=avg,
#                            mode='lines',
#                            opacity=1,
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=6, col=1)
#         if SHOW_alldata:
#             fig.add_trace(go.Scattergl(x=den_df['Date'][::indplot],
#                                  y=den_df['rho (kg/m**3)'][::indplot],
#                        ### name= model_m1,
#                        mode='markers',
#                        opacity=0.5,
#                            marker=dict(color=coldict[model_m1],size=2),
#                            line = dict(dash ='solid', color = coldict[model_m1], width=2),
#                        showlegend=False), row=1, col=1)
        
#         (time_avg, d_avg ) = orbit_avg_generic(den_df['Date'],den_df['rho (kg/m**3)'],
#                                                den_df['Lat'])    
#         fig.add_trace(go.Scattergl(x=time_avg,
#                                  y=d_avg,
#                            mode='lines',
#                            opacity=1,
#                                marker=dict(color=coldict[model_m1],size=1),
#                                line = dict(dash ='solid', color = coldict[model_m1], width=3),
#                            showlegend=False), row=1, col=1)

    

    
    
# #     (time_avg, d_avg ) = orbit_avg_generic(den_dict['dates'],
# #                                    den_dict['den'],
# #                                    den_dict['lat'])
# #     ### -----------------------------------------------------------------------------------------------------
# #     ###     Orbit Averaged Density
# #     fig.add_trace(go.Scattergl(x=time_avg,
# #                                y=d_avg,
# #                                ### name= model_m1,
# #                                mode='markers',
# #                                opacity=opac_val,
# #                                    marker=dict(color= coldict[model_m1],size=2),
# #                                    line = dict(dash ='solid', color = coldict[model_m1], width=2),
# #                                showlegend=False), row=1, col=1)
    
    
# #                 time_avg,CD_avg, area_avg_rolling, CD_std = orb_avg_param(obj_m1.__dict__['DragFile'], arc_string, 'TOTAREA')
# #                 fig.add_trace(go.Scattergl(x=time_avg,
# #                                            y=area_avg_rolling,
# #     #                                          name= plot_name+arc,
# #                                              mode='markers+lines',
# #                                              opacity=1,
# #                                              marker=dict(color=col,size=2,),
# #                                              line = dict(shape='hvh', dash ='solid', color = col, width=3),
# #                                              showlegend=False, ),
# #                                              secondary_y=False,
# #                                              row=3, col=1,
# #                                          )



        

        
#     return(fig)







# fig  = make_subplots(
#     rows=6, cols=1,
# #     subplot_titles=(['Density (kg/m^3)'   , 
# # #                      'CD*A'               ,
# #                      'CD'                 ,   
# #                      'Relative Velocity'  , 
# #                      'Speed Ratio',
# #                     ]),
#     vertical_spacing = 0.05,
# #     horizontal_spacing = 0.05,
#     shared_xaxes=True)



# # run_dict = { }
# # rms_total_return = {}

# for model in run_dict.keys():
# #     for imonth,month in enumerate(month_list):

# #     for ialph,alpha in enumerate(alphas):   
#          # fig,rms_total_return = PLOT__intrack_residuals_w_rms_w_cd(fig, Obj_Geodyn['msis2'] , 0,  arc_listlist)
# #         run_dict[val+"alpha_"+alpha] = ialph

#     fig = PLOT__SATDRAG_STATE(fig, obj[model],   models_dens[model])

    
# #######################################################
# font_dict=dict(family='Arial',size=11,color='black')
# ## automate the specification of the axes for subplots
# rownum, colnum = fig._get_subplot_rows_columns()
# for i in rownum:
#     if len(rownum)==1:
#         L_ticklabel = True
#     else:
#         if i < len(rownum):
#             L_ticklabel = True
#         else:
#             L_ticklabel = True
#     fig.update_xaxes(### LINE at axis border
#                       showline=True,
#                       showticklabels=L_ticklabel,
# #                       tickformat= '%m/%d',
#                       linecolor='black',
#                       linewidth=1,
#                      ### Major ticks
#                       ticks='inside',
#                       tickfont=font_dict,
#                       mirror=True,
# #                       tickwidth=2,
# #                       ticklen=9,
#                       tickcolor='grey',
# #                       tick0="2018-11-9" ,
# #                       dtick=86400000.0*1,    # milliseconds in a day, every 7 days
#                       #### Minor Ticks
#                        minor=dict(
#                          dtick=86400000.0, # milliseconds in a day
#                          tickwidth=1,
#                          ticklen=4,
#                          tickcolor='grey',
#                          ticks='inside'),
#                       ### GRID
#                        gridcolor='gainsboro',
#                        gridwidth=1,
#                        layer='above traces',
#                        tickangle=0,
#                        row=i, col=1)
#     fig.update_yaxes(showline=True,      # add line at x=0
#                          showticklabels=True,
#                          linecolor='black',  # line color
#                          linewidth=1,        # line size
#                      ticks='inside',     # ticks outside axis
#                      tickfont=font_dict, # tick label font
#                      mirror='allticks',  # add ticks to top/right axes
#                      tickwidth=1,      # tick width
#                      tickcolor='black',  # tick color
#                      gridcolor='gainsboro',
#                      gridwidth=1,
#                      layer='above traces',
#                      row=i, col=1)

    
#     fig.update_yaxes(title_text="Density", 
#                      type="log", 
#                      exponentformat= 'power',row=1, col=1)
#     fig.update_yaxes(title_text="CD", 
#                      exponentformat= 'power',row=2, col=1)
#     fig.update_yaxes(title_text="Total Area", 
#                      exponentformat= 'power',row=3, col=1)
#     fig.update_yaxes(title_text="CD*Area", 
#                      exponentformat= 'power',row=4, col=1)

#     fig.update_yaxes(title_text="Rel. Velocity [m/s] ", 
#                      exponentformat= 'power',row=5, col=1)
#     fig.update_yaxes(title_text="Speed Ratio", 
#                      exponentformat= 'power',row=6, col=1)

#     ###
#     ###  DATE on Final x-Axis only
# #     fig.update_xaxes(title="Date",
# #                      row=4, col=1)
# #     fig.update_xaxes(title="Date",
# #                      row=5, col=2)




# fig.update_layout(
# #                   title = '',
#                   autosize=False,    width=1000,    height=1100,
#                   legend= {'itemsizing': 'trace'},
#                   font=font_dict,
#                   plot_bgcolor='white', 
#                  )
# fig.update_annotations(font_size=16)  # Increase size of subplot title



# # fig.add_vrect(x0="2018-10-16 23:00:00" ,
# #               x1="2018-10-17 00:24:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-28 16:00:00" ,
# #               x1="2018-10-28 17:36:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="2018-10-29 10:18:00" ,
# #               x1="2018-10-29 11:42:00" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)
# # fig.add_vrect(x0="" ,
# #               x1="" ,
# #                 fillcolor="red",
# #                 opacity=0.25,
# #                 line_width=0)

# fig.show(config=config)


